In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE118336"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE118336"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE118336.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE118336.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE118336.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"HTA2.0 (human transcriptome array) analysis of control iPSC-derived motor neurons (MN), FUS-H517D-hetero-iPSC-MN, and FUS-H517D-homo-iPSC-MNs"
!Series_summary	"To assess  RNA regulation in the MN possessing mutated FUS-H517D gene."
!Series_summary	"Fused in sarcoma/translated in liposarcoma (FUS) is a causative gene of familial amyotrophic lateral sclerosis (fALS). Mutated FUS causes accumulation of DNA damage stress and stress granule (SG) formation, etc., thereby motor neuron (MN) death. However, key molecular etiology of mutated FUS-dependent fALS (fALS-FUS) remains unclear. Here, Bayesian gene regulatory networks (GRN) calculated by Super-Computer with transcriptome data sets of induced pluripotent stem cell (iPSC)-derived MNs possessing mutated FUSH517D (FUSH517D MNs) and FUSWT identified TIMELESS, PRKDC and miR-125b-5p as ""hub genes"" which influence fALS-FUS GRNs. miR-125b-5p expression up-regulated in FUSH517D MNs, showed opposite correla

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# The dataset seems to be HTA2.0 (human transcriptome array) analysis which implies gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability Analysis

# For trait (ALS status)
# The genotype key (index 1) contains FUS mutation information which is relevant for ALS
trait_row = 1  # 'genotype' contains information about FUS mutation (WT vs H517D)

# For age - Not available in the sample characteristics
age_row = None

# For gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert the FUS genotype to binary trait value (0 for control, 1 for disease)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # FUSWT/WT is control (0), any H517D mutation indicates disease (1)
    if 'FUSWT/WT' in value:
        return 0
    elif 'H517D' in value:  # Either heterozygous or homozygous H517D mutation
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to numeric value"""
    # Age data not available
    return None

def convert_gender(value):
    """Convert gender to binary (0 female, 1 male)"""
    # Gender data not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial validation results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary format is {row_index: [values_for_samples]}
    # We need to transform this into a proper DataFrame
    
    # Sample characteristics from previous step
    sample_chars = {0: ['cell type: iPSC-MN'], 
                    1: ['genotype: FUSWT/WT', 'genotype: FUSWT/H517D', 'genotype: FUSH517D/H517D'], 
                    2: ['time (differentiation from motor neuron precursor): 2 weeks', 
                        'time (differentiation from motor neuron precursor): 4 weeks']}
    
    # Convert the sample characteristics to a DataFrame format
    # First, determine the number of samples from the row with most entries
    num_samples = max(len(values) for values in sample_chars.values())
    
    # Create a DataFrame with rows for each characteristic and columns for each sample
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=range(num_samples))
    
    # Fill in the values where available, leaving NaN for missing values
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            if col_idx < num_samples:
                clinical_data.iloc[row_idx, col_idx] = value
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0], 1: [1.0], 2: [1.0]}
Clinical data saved to: ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE118336.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')

Gene data dimensions: 70523 genes × 60 samples


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# These identifiers appear to be probe IDs (ending with '_st') from an Affymetrix microarray
# They are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze gene identifiers to match between gene expression data and annotation data
# Looking at the gene expression data, we have IDs like 2824546_st
# Looking at the gene annotation data, we need to identify columns containing similar identifiers

# Check the first few rows of gene_data index to see what the probe IDs look like
print("First few probe IDs in gene_data:")
print(gene_data.index[:5])

# Looking at the annotation data, we need to find which column contains the probe IDs that match
# and which column contains the gene symbols for mapping
print("\nAnnotation column names:")
print(gene_annotation.columns)

# Since the probe IDs in gene_data end with "_st" but the annotation shows different format,
# we need to find which annotation columns might contain the relevant mapping information

# 2. Examine gene annotation data to find proper mapping columns
# Display sample rows with relevant columns to help identify the mapping
print("\nSample gene annotation with ID columns:")
print(gene_annotation[['ID', 'gene_assignment']].head(3))

# Based on the examination, we can see that 'ID' contains identifiers (which might need transformation)
# and 'gene_assignment' contains gene symbol information.

# 3. Extract mapping data and apply it
# First, map probe IDs in gene_data to annotation IDs
# Since Affymetrix HTA 2.0 arrays have a specific format, 
# we need to ensure our mapping works between the two datasets

# Create a mapping dataframe from probe IDs to gene symbols
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

print("\nMapping dataframe preview:")
print(mapping_df.head())
print(f"Mapping dataframe shape: {mapping_df.shape}")

# Apply the mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

print("\nAfter normalizing gene symbols:")
print(f"Shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


First few probe IDs in gene_data:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

Annotation column names:
Index(['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop',
       'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot',
       'unigene', 'category', 'locus type', 'notes', 'SPOT_ID'],
      dtype='object')

Sample gene annotation with ID columns:
                ID                                    gene_assignment
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...



Mapping dataframe preview:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...
3  TC01000004.hg.1  OTTHUMT00000007169 // OTTHUMG00000002525 // NU...
4  TC01000005.hg.1  NR_028322 // LOC100132287 // uncharacterized L...
Mapping dataframe shape: (70753, 2)



Gene expression data after mapping:
Shape: (71528, 60)
First few gene symbols:
Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V',
       'A0'],
      dtype='object', name='Gene')

After normalizing gene symbols:
Shape: (24018, 60)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT'],
      dtype='object', name='Gene')



Gene expression data saved to: ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE118336.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
# (Already done in the previous step, but we're keeping this for clarity)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data (already saved in previous step, but we'll keep it for clarity)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data from Step 2
# This is more reliable than reprocessing the original data
try:
    selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    is_trait_available = True
    print(f"Clinical data loaded from {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# Link clinical and genetic data if trait is available
if is_trait_available:
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # 3. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data is unavailable in this dataset.")
    is_biased = True  # Dataset can't be used without trait data
    linked_data = pd.DataFrame()  # Empty DataFrame

# 5. Validate and save cohort info
# If linked_data is empty because trait is not available, use sample IDs from gene data for metadata
sample_df = pd.DataFrame(index=normalized_gene_data.columns) if linked_data.empty else linked_data
    
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased if is_trait_available else None,
    df=sample_df,
    note="Dataset contains iPSC-derived motor neuron gene expression data from FUS-H517D mutation carriers related to ALS."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (24018, 60)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE118336.csv
Clinical data loaded from ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE118336.csv
Clinical data shape: (1, 3)
Clinical data preview: {'0': [0.0], '1': [1.0], '2': [1.0]}
Linked data shape before handling missing values: (63, 24019)
Data shape after handling missing values: (0, 1)
Quartiles for 'Amyotrophic_Lateral_Sclerosis':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Amyotrophic_Lateral_Sclerosis' in this dataset is fine.

Data shape after removing biased features: (0, 1)
Abnormality detected in the cohort: GSE118336. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json
Dataset deemed not usable for associational studies.
